In [2]:
import os
os.chdir('..')

In [133]:
from pathlib import Path
import json
import cv2
import numpy as np
import torch

In [9]:
output_dir = Path('output')
model_name = 'cascade_lr'
model_dir = output_dir/model_name
config_path = model_dir/'config.yaml'
weights_path = model_dir/'model_0007999.pth'

In [19]:
!python train_net.py --eval-only --config-file {str(config_path)} --exp-name cascade_lr MODEL.WEIGHTS {weights_path} OUTPUT_DIR output

2048, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=2048, eps=1e-05)
          )
        )
      )
    )
  )
  (proposal_generator): RPN(
    (rpn_head): StandardRPNHead(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (objectness_logits): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
      (anchor_deltas): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
    )
    (anchor_generator): DefaultAnchorGenerator(
      (cell_anchors): BufferList()
    )
  )
  (roi_heads): CascadeROIHeads(
    (box_pooler): ROIPooler(
      (level_poolers): ModuleList(
        (0): ROIAlign(output_size=(7, 7), spatial_scale=0.25, sampling_ratio=0, aligned=True)
        (1): ROIAlign(output_size=(7, 7), spatial_scale=0.125, sampling_ratio=0, aligned=True)
        (2): ROIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0, aligned=True)
        (3): ROIAlign(output_size=(7, 7), spatial_scale=0.03125, 

# Test 

In [33]:
data_dir = Path('data')/'til_2020'
eval_coco_path = data_dir/'CV_interim_evaluation.json'
val_coco_path = data_dir/'val_clean.json'
img_dir = data_dir/'CV_interim_images'

In [23]:
with eval_coco_path.open('r') as read_file:
    eval_coco = json.load(read_file)

with val_coco_path.open('r') as read_file:
    val_coco = json.load(read_file)

In [24]:
eval_coco.keys()

dict_keys(['images', 'categories'])

In [26]:
eval_coco['categories']

[{'name': 'tops', 'id': 1},
 {'name': 'trousers', 'id': 2},
 {'name': 'outerwear', 'id': 3},
 {'name': 'dresses', 'id': 4},
 {'name': 'skirts', 'id': 5}]

In [25]:
val_coco['categories']

[{'id': 1, 'name': 'tops'},
 {'id': 2, 'name': 'trousers'},
 {'id': 3, 'name': 'outerwear'},
 {'id': 4, 'name': 'dresses'},
 {'id': 5, 'name': 'skirts'}]

In [28]:
eval_coco['images'][0]

{'file_name': '6964.jpg', 'id': 6964}

# Inference

In [31]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

In [72]:
cfg = get_cfg()
cfg.merge_from_file(config_path)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = str(weights_path)

predictor = DefaultPredictor(cfg)

Use original image size?

In [76]:
submissions = []

# For each image
for img_dict in eval_coco['images']:
    file_name = img_dict['file_name']
    img_id = img_dict['id']

    img = cv2.imread(str(img_dir/file_name))

    outputs = predictor(img)
    instances = outputs['instances']
    pred_classes = np.array(instances.pred_classes)
    pred_boxes = instances.pred_boxes
    scores = np.array(instances.scores)

    # For each bounding box
    for i, box in enumerate(instances.pred_boxes):
        box = np.array(box)
        x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
        width = x2 - x1
        height = y2 - y1

        category_id = pred_classes[i] + 1
        score = scores[i]

        sub_dict = {
            'image_id': img_id,
            'category_id': category_id,
            'bbox': [x1, y1, width, height],
            'score': score
        }

        submissions.append(sub_dict)
    
    print(f'Image {img_id} done...')

print('Inference complete.')

In [112]:
file_name = eval_coco['images'][0]['file_name']

In [113]:
img = cv2.imread(str(img_dir/file_name))

In [114]:
outputs = predictor(img)
instances = outputs['instances']
pred_classes = instances.pred_classes
pred_boxes = instances.pred_boxes
scores = instances.scores

In [143]:
scores[0].item()

0.9711182117462158

# Eval COCO

In [1]:
import os
os.chdir('..')

In [2]:
import json
from pathlib import Path
from PIL import Image

In [21]:
eval_coco_path = 'data/til_2020/CV_final_evaluation.json'
eval_img_dir = Path('data')/'til_2020'/'CV_final_images'

In [22]:
with open(eval_coco_path, 'r') as f:
    eval_coco = json.load(f)

In [23]:
eval_coco.keys()

dict_keys(['images', 'categories', 'annotations'])

In [24]:
eval_coco['images'][0]

{'id': 10044, 'file_name': '10044.jpg', 'height': 2965, 'width': 1977}

In [25]:
eval_coco['categories']

[{'name': 'tops', 'id': 1},
 {'name': 'trousers', 'id': 2},
 {'name': 'outerwear', 'id': 3},
 {'name': 'dresses', 'id': 4},
 {'name': 'skirts', 'id': 5}]

## Height and Width for Images

In [13]:
eval_coco['images'][0]

{'id': 10044, 'file_name': '10044.jpg'}

In [15]:
len(eval_coco['images'])

1972

In [14]:
for img_dict in eval_coco['images']:
    file_name = img_dict['file_name']
    img = Image.open(eval_img_dir/file_name)
    img_dict['height'] = img.height
    img_dict['width'] = img.width

## Dummy Annotations

In [19]:
eval_coco['annotations'] = []

## Save

In [20]:
with open(eval_coco_path, 'w') as f:
    json.dump(eval_coco, f)